# Generate a focus matrix with a single stuttered image
This notebook will take a stuttered image with sub images \
at varied focus positions. \
See SITCOM-932 \
Craig Lage - 30Jun23

In [ ]:
import sys, asyncio, time, os
import numpy as np
from astropy.time import Time, TimeDelta
from astropy import units as u

from lsst.ts import salobj

from lsst.ts.observatory.control.auxtel.atcs import ATCS
from lsst.ts.observatory.control.auxtel.latiss import LATISS
from lsst.ts.observatory.control.utils import RotType

In [ ]:
stream_handler = logging.StreamHandler(sys.stdout)
# if you want logging
logger = logging.getLogger()
logger.addHandler(stream_handler)
logger.level = logging.DEBUG

### Instantiate the control classes

In [ ]:
domain = salobj.Domain()
atcs = ATCS(domain)
latiss = LATISS(domain)
await asyncio.gather(atcs.start_task, latiss.start_task)

## Assumes ATCS and LATISS are enabled and the telescope has run prepare_for_on_sky.

# Find a bright target

I think m=8.0 is about right, but this may take some testing.

In [ ]:
target_name = await atcs.find_target(az=90.0,el=70,mag_limit=8.0)
print(target_name)

# Now slew to the object
### This will also set the correct rotator angle.

In [ ]:
await atcs.slew_object(target_name, rot=0.0, rot_type=RotType.PhysicalSky)

# Now take an image.
### Verify that the star is centered in the field.  If not, center as needed.

In [ ]:
await latiss.take_object(exptime=2.0, n=1, filter='SDSSr_65mm' ,grating='empty_1', reason='SITCOM-932')

# Now we have to offset the star to center it in the first stripe.
It will require moving the image slightly to get it positioned. \
Ideally, it should be 50 pixels (5.0 arcseconds) above the midline. \
The step below should do this if it is perfectly centered to begin with. \
Then take a new image to verify \
This may take a few iterations to get it in the right place.


In [ ]:
await atcs.offset_xy(y=33.0, x=0)
await latiss.take_object(exptime=2., n=1, filter='SDSSr_65mm',grating='empty_1', reason='SITCOM-932')

In [ ]:
await latiss.take_object(exptime=2., n=1, filter='SDSSr_65mm',grating='empty_1', reason='SITCOM-932')

# Now try taking the stuttered focus matrix.
As currently configured, there will be 20 100 pixel strips. \
The even numbered strips will be the ones to be used for building the focus parabola. \
There will be 9 of these running from z_offset_start to z_offset_end. \
The odd numbered strips are where the focus is being shifted and should not be used for building the focus parabola.

In [ ]:
z_offset_start = -0.1 # mm
z_offset_step = 0.025 # mm
z_offset_end = -z_offset_start
n_focus_shifts = int(abs(z_offset_start / z_offset_step)) * 2 + 1
# Shift the focus in the negative direction
await atcs.rem.ataos.cmd_offset.set_start(z=z_offset_start)
total_focus = z_offset_start
await asyncio.sleep(2)

exptime = 2.0
expcount = 20
rowshift = 100
buffer = 30.0 # Time buffer for timeout
timeout = exptime * expcount + buffer

await latiss.rem.atcamera.cmd_enableCalibration.set_start()
keyValueMap = f"groupId: {Time.now().isot},imageType: ENGTEST"
print(keyValueMap)
latiss.rem.atcamera.cmd_startImage.set(shutter=True, keyValueMap=keyValueMap, timeout=timeout)
await latiss.rem.atcamera.cmd_startImage.start()
n_shifts = 0
for i in range(expcount - 1):
    if ((i + 1) % 2 == 0) and (n_shifts < n_focus_shifts - 1):
        print(f"Shifting focus by {z_offset_step}")
        total_focus += z_offset_step
        await atcs.rem.ataos.cmd_offset.set_start(z=z_offset_step)
        n_shifts += 1
    print(f"Exposing {exptime} seconds. Focus = {total_focus:.3f}")
    await asyncio.sleep(exptime)
    latiss.rem.atcamera.cmd_discardRows.set(nRows=rowshift)
    print(f"Shifting {rowshift} rows.")
    await latiss.rem.atcamera.cmd_discardRows.start()
await asyncio.sleep(exptime)
await latiss.rem.atcamera.cmd_endImage.start()
await latiss.rem.atcamera.cmd_disableCalibration.start()
# Move back to the original focus offset position
await atcs.rem.ataos.cmd_offset.set_start(z= -z_offset_end)


In [ ]:
z_offset_start = -0.2 # mm
z_offset_step = 0.05 # mm
z_offset_end = -z_offset_start
n_focus_shifts = int(abs(z_offset_start / z_offset_step)) * 2 + 1
# Shift the focus in the negative direction
await atcs.rem.ataos.cmd_offset.set_start(z=z_offset_start)
total_focus = z_offset_start
await asyncio.sleep(2)

exptime = 2.0
expcount = 20
rowshift = 100
buffer = 30.0 # Time buffer for timeout
timeout = exptime * expcount + buffer

await latiss.rem.atcamera.cmd_enableCalibration.set_start()
keyValueMap = f"groupId: {Time.now().isot},imageType: ENGTEST"
print(keyValueMap)
latiss.rem.atcamera.cmd_startImage.set(shutter=True, keyValueMap=keyValueMap, timeout=timeout)
await latiss.rem.atcamera.cmd_startImage.start()
n_shifts = 0
for i in range(expcount - 1):
    if ((i + 1) % 2 == 0) and (n_shifts < n_focus_shifts - 1):
        print(f"Shifting focus by {z_offset_step}")
        total_focus += z_offset_step
        await atcs.rem.ataos.cmd_offset.set_start(z=z_offset_step)
        n_shifts += 1
    print(f"Exposing {exptime} seconds. Focus = {total_focus:.3f}")
    await asyncio.sleep(exptime)
    latiss.rem.atcamera.cmd_discardRows.set(nRows=rowshift)
    print(f"Shifting {rowshift} rows.")
    await latiss.rem.atcamera.cmd_discardRows.start()
await asyncio.sleep(exptime)
await latiss.rem.atcamera.cmd_endImage.start()
await latiss.rem.atcamera.cmd_disableCalibration.start()
# Move back to the original focus offset position
await atcs.rem.ataos.cmd_offset.set_start(z= -z_offset_end)
